In [3]:
#importing liberaries
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
import pathlib
from keras.layers import BatchNormalization
from tensorflow.keras import Sequential
import tensorflow_datasets as tfds
import os
import cv2
import matplotlib.pyplot as plot
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam
from os import listdir
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [10]:
EPOCHS = 30
INIT_LR = 1e-1
Batch_size =32
Image_size = (128,128)
directory_root = '/home/zesam/Downloads/My implentation/Flower classification using ALexNet/flowers'
default_image_size = tuple((128, 128))

In [11]:
files = os.listdir(directory_root)
print(files)

['daisy', 'sunflower', 'rose', 'dandelion', 'tulip']


In [12]:
#creating image list containing all Dataset images as directories
image_dir_list = []
label_list = []
root_dir = listdir(directory_root)
for plant_disease_folder in root_dir:
    plant_disease_dir = listdir(f"{directory_root}/{plant_disease_folder}")
    for image in plant_disease_dir:
        plant_image_dir = f"{directory_root}/{plant_disease_folder}/{image}"
        if plant_image_dir.endswith(".jpg") == True or plant_image_dir.endswith(".JPG") == True:
            image_dir_list.append(plant_image_dir)
            label_list.append(plant_disease_folder)

In [13]:
#importing all images in a list
images_list= []
for image_dir in image_dir_list:
    if image_dir.endswith(".jpg") == True or image_dir.endswith(".JPG") == True:
        image = cv2.imread(image_dir)
        image = cv2.resize(image, default_image_size)   
        image_array = img_to_array(image)
        images_list.append(image_array)

In [14]:
#Converting images to numpy array with normalization
np_image_list = np.array(images_list, dtype=np.float16) / 225.0
print(np_image_list.shape)


(4317, 128, 128, 3)


In [15]:
#binarizing labels
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
n_classes = len(label_binarizer.classes_)


In [16]:
X = np_image_list
y = image_labels


In [17]:
#test lists
accuracy_test = []
#Train lists
accuracy_train = []

In [19]:
from sklearn.model_selection import KFold
k = 3

kf = KFold(n_splits= k, random_state=True, shuffle=True)
for train_index, test_index in kf.split(X) :
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    np.random.seed(1000)

    #Instantiation
    AlexNet = Sequential()

    #1st Convolutional Layer
    AlexNet.add(Conv2D(filters=96, input_shape=(128,128,3), kernel_size=(11,11), strides=(4,4), padding ='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #2nd Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #3rd Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))

    #4th Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))

    #5th Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #Passing it to a Fully Connected layer
    AlexNet.add(Flatten())
    # 1st Fully Connected Layer
    AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    AlexNet.add(Dropout(0.4))

    #2nd Fully Connected Layer
    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

   #3rd Fully Connected Layer
    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

    #Output Layer
    AlexNet.add(Dense(5))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))
    AlexNet.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0003), loss='binary_crossentropy', metrics=["accuracy"])
   
    #training model
    
    AlexNet.fit(X_train,y_train,epochs = 7)
    #model prediction 
  
    Model_pred_train = AlexNet.predict(X_train)
  
    Model_pred_test =  AlexNet.predict(X_test)

    
    Model_pred_test = Model_pred_test > 0.5
    Model_pred_train = Model_pred_train > 0.5
    
 
    ################################################33
    print("Test Evaluation")
    print("="*40)
    #Calculating Test accuracy
    print('Test accuracy is :' + str(accuracy_score(y_test,Model_pred_test)*100))
    accuracy = accuracy_score(y_test,Model_pred_test)*100
    accuracy_test.append(accuracy)
    ################################################################
    print("="*40)
    print("Train Evaluation")
    print("="*40)
    print('Training accuracy is : ' + str(accuracy_score(y_train,Model_pred_train)*100))
    t_accuracy = accuracy_score(y_train,Model_pred_train)*100
    accuracy_train.append(t_accuracy)  
    #################################################################
    print("="*40)
    #model prediction 
  
    Model_pred_train = AlexNet.predict(X_train)
  
    Model_pred_test =  AlexNet.predict(X_test)

    
    Model_pred_test = Model_pred_test > 0.5
    Model_pred_train = Model_pred_train > 0.5
    
 
    ################################################33
    print("Test Evaluation")
    print("="*40)
    #Calculating Test accuracy
    print('Test accuracy is :' + str(accuracy_score(y_test,Model_pred_test)*100))
    accuracy = accuracy_score(y_test,Model_pred_test)*100
    accuracy_test.append(accuracy)
    ################################################################
    print("="*40)
    print("Train Evaluation")
    print("="*40)
    print('Training accuracy is : ' + str(accuracy_score(y_train,Model_pred_train)*100))
    t_accuracy = accuracy_score(y_train,Model_pred_train)*100
    accuracy_train.append(t_accuracy)  
    #################################################################
    print("="*40)

Epoch 1/7
90/90 [==============================] - 133s 1s/step - loss: 0.6286 - accuracy: 0.4861
Epoch 2/7
90/90 [==============================] - 113s 1s/step - loss: 0.5772 - accuracy: 0.5605
Epoch 3/7
90/90 [==============================] - 111s 1s/step - loss: 0.5402 - accuracy: 0.6352
Epoch 4/7
90/90 [==============================] - 106s 1s/step - loss: 0.5211 - accuracy: 0.6605
Epoch 5/7
90/90 [==============================] - 105s 1s/step - loss: 0.4959 - accuracy: 0.6932
Epoch 6/7
90/90 [==============================] - 105s 1s/step - loss: 0.4821 - accuracy: 0.7099
Epoch 7/7
90/90 [==============================] - 105s 1s/step - loss: 0.4559 - accuracy: 0.7543
Test Evaluation
Test accuracy is :49.61779013203614
Train Evaluation
Training accuracy is : 55.90687977762335
Test Evaluation
Test accuracy is :49.61779013203614
Train Evaluation
Training accuracy is : 55.90687977762335


/home/zesam/.local/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/7
90/90 [==============================] - 146s 1s/step - loss: 0.6179 - accuracy: 0.5160
Epoch 2/7
90/90 [==============================] - 109s 1s/step - loss: 0.5695 - accuracy: 0.5813
Epoch 3/7
90/90 [==============================] - 106s 1s/step - loss: 0.5439 - accuracy: 0.6258
Epoch 4/7
90/90 [==============================] - 112s 1s/step - loss: 0.5125 - accuracy: 0.6727
Epoch 5/7
90/90 [==============================] - 110s 1s/step - loss: 0.4918 - accuracy: 0.7113
Epoch 6/7
90/90 [==============================] - 109s 1s/step - loss: 0.4687 - accuracy: 0.7436
Epoch 7/7
90/90 [==============================] - 106s 1s/step - loss: 0.4495 - accuracy: 0.7669
Test Evaluation
Test accuracy is :48.92286309937457
Train Evaluation
Training accuracy is : 55.03822098679638
Test Evaluation
Test accuracy is :48.92286309937457
Train Evaluation
Training accuracy is : 55.03822098679638


/home/zesam/.local/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/7
90/90 [==============================] - 106s 1s/step - loss: 0.6219 - accuracy: 0.5073
Epoch 2/7
90/90 [==============================] - 105s 1s/step - loss: 0.5731 - accuracy: 0.5837
Epoch 3/7
90/90 [==============================] - 105s 1s/step - loss: 0.5425 - accuracy: 0.6345
Epoch 4/7
90/90 [==============================] - 105s 1s/step - loss: 0.5174 - accuracy: 0.6689
Epoch 5/7
90/90 [==============================] - 105s 1s/step - loss: 0.4948 - accuracy: 0.6904
Epoch 6/7
90/90 [==============================] - 105s 1s/step - loss: 0.4751 - accuracy: 0.7300
Epoch 7/7
90/90 [==============================] - 105s 1s/step - loss: 0.4511 - accuracy: 0.7620
Test Evaluation
Test accuracy is :50.590687977762336
Train Evaluation
Training accuracy is : 61.257817929117444
Test Evaluation
Test accuracy is :50.590687977762336
Train Evaluation
Training accuracy is : 61.257817929117444


In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk
BaymaxLittleScreen = tk.Tk()
BaymaxLittleScreen.geometry("400x400")  # Size of the window 
BaymaxLittleScreen.title('Dr.BayMax the intern')
my_font1=('times', 18, 'bold')
my_font2 = ('italic',10 , 'bold')

#DR.BayMax Label
l1 = tk.Label(BaymaxLittleScreen,text='Dr.BayMax',width=30,font=my_font1)  
l1.grid(row=1,column=1)
#Upload file Button
b1 = tk.Button(BaymaxLittleScreen, text='Upload File', width=20,command = lambda:upload_file())
b1.grid(row=2,column=1)
#show predictions 
b3 = tk.Button(BaymaxLittleScreen, text='Show Classification', width=20,command = lambda:ShowClassification())
b3.grid(row=8,column=1)
l2 = tk.Label(BaymaxLittleScreen,text='Classification',width=30,font=my_font1)  
l2.grid(row=10,column=1)



def upload_file():
    global img
    f_types = [('Jpg Files', '*.jpg')]
    global filename
    filename = filedialog.askopenfilename(filetypes=f_types)
    #Preview Image on the Pane 
    img = ImageTk.PhotoImage(file=filename)
    b2 =tk.Button(BaymaxLittleScreen,image=img) # using Button 
    b2.grid(row=3,column=1)
    global image_to_Save 
    #creating image object to create their directory
    image_to_Save = Image.open(filename)
    image_To_Save = open_file(image_to_Save) #Whatever the given image is    
    
    
def open_file(self):
    curr_directory = os.getcwd()
    self.file_path = filename
    save_path = "/home/zesam/Downloads/My implentation/Skin Disease Detection using CNN"
    file_name = "test.jpg"
    global complete_name
    complete_name = os.path.join(save_path, file_name)
    image_to_Save.save(complete_name)
    if self.file_path is not None:
        pass    


def ShowClassification():
    img_path = complete_name
    image = tf.keras.preprocessing.image.load_img(img_path, target_size=(96, 96))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    input_arr = input_arr.astype('float32') / 255.0
    prediction = AlexNet.predict(input_arr)
    if(prediction == 1 ):
        l3 = tk.Label(BaymaxLittleScreen,text='Vtiligo Case',width=30,font= my_font2)
        l3.grid(row=12,column=1) 
    else:
        l4 = tk.Label(BaymaxLittleScreen,text='Healthy skin',width=30,font= my_font2)
        l4.grid(row=14,column=1)
        
    plot.imshow(prediction, interpolation='nearest')
    plot.show()
BaymaxLittleScreen.mainloop()  # Keep the window open

In [ ]:
test_accuracy_avg = 0
for accuracy in accuracy_test:
    test_accuracy_avg += accuracy
test_accuracy_average = test_accuracy_avg/k
print("Accuracy array: " + str(accuracy_test))
print("Average Accuracy : " + str(test_accuracy_average))



In [ ]:
train_accuracy_avg = 0
for train_accuracy in accuracy_train:
    train_accuracy_avg += train_accuracy
train_accuracy_average = train_accuracy_avg/k
print(accuracy_train)
print("="*40)
print("Train accuracy average: "+ str(train_accuracy_average))
